In [1]:
import pandas as pd
import numpy as np
from time import time
import re
from tqdm import tqdm
import missingno as msno
from google.cloud import bigquery


In [2]:
bqclient = bigquery.Client(project='charged-genre-350106')

### 1. 제주패스 차종 데이터 전처리

In [3]:
query_rentcar= """
SELECT
  product_name,
  car_type
FROM 
  `charged-genre-350106.bcml.bcml_rentcar`
"""

In [4]:
jjps= bqclient.query(query_rentcar).to_dataframe()

In [5]:
#null값 제거
jjps=jjps.dropna(axis=0)

#중복 자동차 이름 
jjps=jjps.drop_duplicates(['product_name'])

#리인덱싱(중간에 빈 인덱스)
jjps=jjps.reset_index()

#괄호 및 괄호안의 문자 제거 
for i in tqdm(range(len(jjps))):
    jjps['product_name'][i]=re.sub('\(.*\)|\s-\s.*','',jjps['product_name'][i])

#필요 데이터만 남기고
jjps=jjps[['product_name','car_type']]

#컬럼명 수정
jjps.columns=['car_name','car_type']

# 차종 코드별로 통합
jjps=jjps.groupby('car_type')['car_name'].apply(' '.join).to_frame().reset_index()

#불용어 정리 for 제주패스 차종
stop_words="더 뉴 넥스트 인스퍼레이션 올 어반 차박 캠핑 컨버터블 컨버터블S 라이즈 LF DN8 1세대 3세대 1.6 터보 8세대 2세대 \
            4세대 4인 5인 6인 7인 8인 9인 11인 12인 15인 TM 하이리무진 스포츠 2.0 718 250 마스터 티 R AWD 4륜 2세대 3세대 휘 LPG 휘발유 \
            가솔린 경유 프리미엄 플래티넘 올뉴 더뉴 LE 19 2015 2016 2017 2018 2019 2020 2021 2022 만26세 9인승 AWD │ 2500 초이스 AT 2 \
            The II AD G 147호1005 │105허1117 L D 24호2412 디젤2WD THE 22 디젤 디럭스 New 1600 VGT 디럭스 105하9373\
            713하2001 138하1356 41하8080 63호0394 7인20 109호1363 렌터카 모던 스탠다드 202021 2WD 138하1195 43호0974 2111 NEW ALL 22 14 T 25 All new 프리미어 \
            20182020 엠디 201618 16 20162021 202122 3세대0 55허8203 162호1003 201718 2018더뉴 엘피지 1821 모자이카 긴출 21년휘발유 19년 1719 GTe 2세대2022년 \
            LPG2021 7인승 16T 7인승경 컨버터블자차 21년 1920LPG 1920년 럭셔리휘 1819 1819휘발유 2021년 1819경유 19년휘발유 20년휘발유 22년 휘발유 2021년휘발유 5새대가솔린2WD \
            19년LPG 아이소픽스 썬루프 Lpi 롱 레인지 퍼포먼스 4도어 롱레인지 C GLE GLB 5도어 제타 이모션 5세대 플레티넘 저상 휠체어 슬로프 투어 라운지 수동 마스터 네오 MASTER X ev EV"

# 불용어 제거 테스트
# stop_words = set(stop_words.split(' '))
# word_tokens = ''.join((jjps['car_name'][0])).split()
# result = [word for word in word_tokens if not word in stop_words]
# print('불용어 제거 전 :',word_tokens) 
# print('불용어 제거 후 :',result)

  0%|          | 0/393 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":
100%|██████████| 393/393 [00:00<00:00, 3757.87it/s]


In [6]:
for i in tqdm(range(len(jjps))):
    word_tokens = ''.join((jjps['car_name'][i])).split()
    result = [word for word in word_tokens if not word in stop_words]
    jjps['car_name'][i]=result

100%|██████████| 9/9 [00:00<00:00, 1854.61it/s]


In [7]:
jjps

,car_type,car_name
0,01,"[스파크, 모닝, 캐스퍼, 모닝, 캐스퍼, 스파크, 모닝, 모닝, 모닝, 모닝, 모..."
1,02,"[엑센트, 볼트, 미니, 쿠퍼, 엑센트, CLIO, 미니, 쿠퍼, 엑센트, 푸조, ..."
2,03,"[SM6, 말리부, 쏘나타, 벤츠, E220D, 카브리올레, 테슬라모델3, 쏘나타,..."
3,04,"[그랜저, IG, 포드, 머스탱, 그랜저IG, 지프, 랭글러, 루비콘, 제네시스, ..."
4,05,"[지프, 레니게이드, 팰리세이드, 코나, 쏘렌토, 쏘렌토, 쏘렌토, 스포티지, 쏘렌..."
5,06,"[카니발, 카니발, 카니발, 스타리아, 카니발, 카니발, 스타리아, 그랜드, 스타렉..."
6,07,"[SM3, 아반떼CN7, 토요타, 프리우스, 미니, 쿠퍼, SM3, ZE, K3, ..."
7,08,"[코란도, 니로, 제네시스, GV60, 아이오닉6, BMW, I4]"
8,09,"[미니, 쿠퍼, 컨버터블JCW, 미니, 쿠퍼, 미니, 쿠퍼, 컨트리맨, BMW, 4..."


In [8]:
#중복제거 루프
for r in range(len(jjps)):
    a=[]
    for value in jjps['car_name'][r]:
        if value not in a:
            a.append(value)
            jjps['car_name'][r]=a

#소문자를 대문자로 변환
for k in range(len(jjps)):
    upper=[]
    for l in jjps['car_name'][k]:
        upper.append(l.upper())
        jjps['car_name'][k]=upper

In [9]:
# 제주패스 차종 데이터 전처리 완료
jjps

,car_type,car_name
0,01,"[스파크, 모닝, 캐스퍼]"
1,02,"[엑센트, 볼트, 미니, 쿠퍼, CLIO, 푸조, E208]"
2,03,"[SM6, 말리부, 쏘나타, 벤츠, E220D, 카브리올레, 테슬라모델3, 쏘나타D..."
3,04,"[그랜저, IG, 포드, 머스탱, 그랜저IG, 지프, 랭글러, 루비콘, 제네시스, ..."
4,05,"[지프, 레니게이드, 팰리세이드, 코나, 쏘렌토, 스포티지, 볼보, XC40, 싼타..."
5,06,"[카니발, 스타리아, 그랜드, 스타렉스, 쏠라티, 벤츠, 스프린터, 뉴카운티, 코란..."
6,07,"[SM3, 아반떼CN7, 토요타, 프리우스, 미니, 쿠퍼, ZE, K3, I30, ..."
7,08,"[코란도, 니로, 제네시스, GV60, 아이오닉6, BMW, I4]"
8,09,"[미니, 쿠퍼, 컨버터블JCW, 컨트리맨, BMW, 420I, 에스컬레이드, 포르쉐..."


In [10]:
# 데이터 확인
# jjps.to_csv('jjps_final.csv', encoding='euc-kr')

### 2. ERP 차종 데이터 전처리

In [11]:
query_erp= """
SELECT
  model_name
FROM 
  `charged-genre-350106.bcml.bcml_erp`
"""

In [12]:
# 데이터 로드
erp = bqclient.query(query_erp).to_dataframe()

# 중복 데이터 제거
erp=erp.drop_duplicates(['model_name'])

# 리인덱싱(중간에 빠진 인덱스넘버 제거)
erp=erp.reset_index()

# 특수문자 및 괄호 및 괄호 안 문자 제거
for i in tqdm(range(len(erp))):
    erp['model_name'][i]=re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]','',erp['model_name'][i])
    erp['model_name'][i]=re.sub('\(.*\)|\s-\s.*','',erp['model_name'][i])

# 필요한 컬럼 선택
erp=erp[['model_name']]

  0%|          | 0/1200 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == "":
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
100%|██████████| 1200/1200 [00:00<00:00, 1825.76it/s]


In [13]:
erp['model_name']=erp['model_name'].str.upper()

In [14]:
##소나타 => 쏘나타, 아반테 => 아반떼, 그랜져=>그랜저, PEUGEOT=>푸조, 산타페=>싼타페, 시트로앵=>시트로엥, RAY=>레이,테슬라=>테슬라모델3, 뉴라이즈=>쏘나타,아이오니=>아이오닉,올뉴카=>카니발, PRIUS=>프리우스
##펠리세이드 => 팰리세이드, AD =>아반떼, CN7=>아반떼, LF>쏘나타, DL3=>K5, DN8=>쏘나타,28=>2008(푸조),50=>5008(푸조), rise=>쏘나타,아이써티=>I30

In [15]:
for i in range(len(erp)):
    if '소나타' in erp['model_name'][i]:
        erp['model_name'][i]='쏘나타'
    elif '아반테' in erp['model_name'][i]:
        erp['model_name'][i]='아반떼'
    elif '그랜져' in erp['model_name'][i]:
        erp['model_name'][i]='그랜저'
    elif 'PEUGEOT' in erp['model_name'][i]:
        erp['model_name'][i]='푸조'
    elif '산타페' in erp['model_name'][i]:
        erp['model_name'][i]='싼타페'
    elif '시트로앵' in erp['model_name'][i]:
        erp['model_name'][i]='시트로엥'
    elif 'RAY' in erp['model_name'][i]:
        erp['model_name'][i]='레이'
    elif '테슬라' in erp['model_name'][i]:
        erp['model_name'][i]='테슬라모델3'
    elif '뉴라이즈' in erp['model_name'][i]:
        erp['model_name'][i]='쏘나타'
    elif '아이오니' in erp['model_name'][i]:
        erp['model_name'][i]='아이오닉'
    elif '올뉴카' in erp['model_name'][i]:
        erp['model_name'][i]='카니발'
    elif 'PRIUS' in erp['model_name'][i]:
        erp['model_name'][i]='프리우스'
    elif '펠리세이드' in erp['model_name'][i]:
        erp['model_name'][i]='팰리세이드'
    elif 'AD' in erp['model_name'][i]:
        erp['model_name'][i]='아반떼'
    elif 'CN7' in erp['model_name'][i]:
        erp['model_name'][i]='아반떼'
    elif 'LF' in erp['model_name'][i]:
        erp['model_name'][i]='쏘나타'
    elif 'DL3' in erp['model_name'][i]:
        erp['model_name'][i]='K5'
    elif 'DN8' in erp['model_name'][i]:
        erp['model_name'][i]='쏘나타'
    # elif '2008' in erp['model_name'][i]:
    #     erp['model_name'][i]='푸조 2008'
    # elif '5008' in erp['model_name'][i]:
    #     erp['model_name'][i]='푸조 5008'
    # elif '3008' in erp['model_name'][i]:
    #     erp['model_name'][i]='푸조 3008'   
    elif 'rise' in erp['model_name'][i]:
        erp['model_name'][i]='쏘나타'
    elif '아이써티' in erp['model_name'][i]:
        erp['model_name'][i]='I30'

### 3. ERP & 제주패스 차종 Searcing & Mapping

In [16]:
# 제주패스 차종 기준으로 ERP데이터 차종코드 컬럼 추가 (차종 코드 여러개가 있음) 
dict_ = {x: [] for x in range(len(erp))}
for i in tqdm(range(len(jjps))):
    #type=[]
    jjps_dic=jjps.iloc[i].to_dict()
    for j in range(len(jjps_dic['car_name'])):
        # type=[]
        for k in range(len(erp['model_name'])):
            erp_dic=erp.iloc[k].to_dict()
            #type=[]
            if jjps_dic['car_name'][j] in erp_dic['model_name']:
                dict_[k].append(jjps_dic['car_type'])
                # type.append(jjps_dic['car_type'])
                # erp['car_type'].iloc[k]=type
dict_ = {k:list(set(v)) for k, v in dict_.items()}

100%|██████████| 9/9 [00:16<00:00,  1.84s/it]


In [17]:
# 딕셔너리 to 데이터프레임
df=pd.DataFrame.from_dict(dict_, orient='index')

# 빈컬럼 만들고
df['list']=""

# 빈컬럼에 차종 코드 리스트로
for i in range(len(df)):
    df['list'][i]=df.loc[i,:5].values.tolist()

#차종 코드 리스트 to string변환 
df['list']=df['list'].apply(lambda x: ','.join(map(str, x)))

In [18]:
#'07'코드 포함 인덱스 선택 확인
df[df['list'].str.contains('07')]

,0,1,2,3,4,5,list
0,07,None,None,None,None,None,"07,None,None,None,None,None"
2,07,None,None,None,None,None,"07,None,None,None,None,None"
3,07,None,None,None,None,None,"07,None,None,None,None,None"
5,07,None,None,None,None,None,"07,None,None,None,None,None"
7,07,None,None,None,None,None,"07,None,None,None,None,None"
...,...,...,...,...,...,...,...
1178,07,None,None,None,None,None,"07,None,None,None,None,None"
1184,07,None,None,None,None,None,"07,None,None,None,None,None"
1187,07,None,None,None,None,None,"07,None,None,None,None,None"
1191,07,None,None,None,None,None,"07,None,None,None,None,None"


In [19]:
erp

,model_name
0,K3
1,K5
2,K3
3,SM3
4,19K5
...,...
1195,쏘나타 뉴 라이즈 L 2018 │ 24호2412
1196,쏘렌토 더 마스터 D 2019 │105허1117
1197,아반떼
1198,2020 THE K9 3800 플래티넘 II AT 휘발유 2020


In [20]:
df

,0,1,2,3,4,5,list
0,07,None,None,None,None,None,"07,None,None,None,None,None"
1,03,None,None,None,None,None,"03,None,None,None,None,None"
2,07,None,None,None,None,None,"07,None,None,None,None,None"
3,07,None,None,None,None,None,"07,None,None,None,None,None"
4,03,None,None,None,None,None,"03,None,None,None,None,None"
...,...,...,...,...,...,...,...
1195,03,None,None,None,None,None,"03,None,None,None,None,None"
1196,05,None,None,None,None,None,"05,None,None,None,None,None"
1197,07,None,None,None,None,None,"07,None,None,None,None,None"
1198,04,None,None,None,None,None,"04,None,None,None,None,None"


In [21]:
# erp에 맵핑된 차종코드 컬럼만 선택
df=df[['list']]

# erp와 df 맵핑
erp_type=pd.merge(erp, df, left_index=True, right_index=True)

#데이터 확인
erp_type
#erp_type.to_csv('erp_final.csv', encoding='euc-kr')

,model_name,list
0,K3,"07,None,None,None,None,None"
1,K5,"03,None,None,None,None,None"
2,K3,"07,None,None,None,None,None"
3,SM3,"07,None,None,None,None,None"
4,19K5,"03,None,None,None,None,None"
...,...,...
1195,쏘나타 뉴 라이즈 L 2018 │ 24호2412,"03,None,None,None,None,None"
1196,쏘렌토 더 마스터 D 2019 │105허1117,"05,None,None,None,None,None"
1197,아반떼,"07,None,None,None,None,None"
1198,2020 THE K9 3800 플래티넘 II AT 휘발유 2020,"04,None,None,None,None,None"


In [22]:
#푸조 제거
erp_type_np=erp_type.drop(erp_type[(erp_type['model_name'].str.contains('푸조'))].index)

# 푸조 제거됐는 확인
erp_type_np[(erp_type_np['model_name'].str.contains('푸조'))]

,model_name,list


In [23]:
# 트레일블레이저를 제외, 01 경차 차종 차량명 선택
type_01=pd.DataFrame(erp_type_np[(erp_type_np['list'].str.contains('01')) & (erp_type_np['model_name'].str.contains('트레')==False)].model_name)

# 데이터 확인
list(type_01.model_name)

['모닝',
 '더뉴 스파크',
 '더뉴모닝휘',
 '올 뉴 모닝0',
 '넥스트스파크',
 '1821 올뉴 모닝',
 '올뉴모닝 19년',
 '모닝 휘발유 2017',
 '올뉴모닝럭셔리',
 '캐스퍼휘4인승',
 '신형 모닝2022년',
 '더뉴 스파크 2021년',
 '캐스퍼 인스퍼레이션',
 '캐스퍼 2122년휘발유',
 '모닝 어반 20년휘발유',
 '스파크',
 '캐스퍼',
 ' 스파크',
 '더뉴모닝',
 '모닝어반',
 '올뉴모닝',
 '모닝 어반',
 '모닝어반',
 '올뉴 모닝',
 '1819올뉴모닝',
 '더 뉴 스파크',
 '모닝 가솔린',
 '더뉴스파크휘',
 '올뉴모닝  긴출',
 '더뉴스파크2021년',
 '모닝 어반 휘발유',
 '모닝어반2022  만21세',
 '올뉴모닝 럭셔리휘',
 '올뉴모닝 휘발유 2019',
 'K 캐스퍼 22년휘발유',
 '모닝 어반 2022',
 '모닝 휘발유 2020',
 '모닝 10 G',
 '더뉴모닝 휘발유 201617',
 '올뉴모닝 휘발유 2016',
 '더뉴스파크',
 '모닝 20162021',
 '모닝 4',
 '모닝2021',
 '모닝 가솔린',
 '모닝 휘발유 2017 ']

### 차종별 데이터 확인하고 고민 및 해야할 일 정리

In [24]:
#01 경형     : 분석가능, 먼저 분석 해보고 다른 차종 데이터 정리 
#02 소형     : 수입차(미니쿠퍼, 볼트) 제외, 가격에 차이가 있을 것으로 판단하기 때문, 수입 차종 별도로 분석
#03 중형     : 수입차(테슬라/BMW/BENZ/벤츠) 제외,가격에 차이가 있을 것으로 판단하기 때문, k7은 고급으로 봐야하지 않을까?
#04 고급     : k7은 고급으로 봐야하지 않을까?
#05 SUV/캠핑 : 수입(BMW520 등) 제외, SUV도 소형/중형/대형으로 분류필요
#06 승합     : 수입(벤츠C200카브리올레,벤츠 220D 등) 제외
#07 준중형    : 수입제외
#08 전기     : 전기는 분석하지 않음(비중이 많지 않기 때문) 만약에 한다면 소형 중형 대형으로 분류필요
#09 수입     : 수입차만 별도로, 소형/중형/대형 분류필요

In [26]:
type_01['car_type']='01'
type_01

,model_name,car_type
10,모닝,01
101,더뉴 스파크,01
113,더뉴모닝휘,01
116,올 뉴 모닝0,01
128,넥스트스파크,01
133,1821 올뉴 모닝,01
148,올뉴모닝 19년,01
160,모닝 휘발유 2017,01
164,올뉴모닝럭셔리,01
166,캐스퍼휘4인승,01


In [28]:
type_01.to_csv('type_01.csv', encoding='euc-kr')